In [90]:
import pandas as pd
import pymysql
from datetime import datetime
import numpy as np

# 加载CSV文件
csv_file = 'data_portfolio_24.csv'
df = pd.read_csv(csv_file)

# 数据库连接信息
host = 'csmysql.cs.cf.ac.uk'
user = 'c23042881'
password = 'WYF990816sq'
db = 'c23042881_c23042881_database1'

# 将Unix时间戳转换为MySQL的DATETIME格式的函数
def convert_timestamp(row):
    if pd.isnull(row) or np.isnan(row):  # 检查是否为NaN
        return None  # 如果是NaN，返回None
    try:
        # 尝试将时间戳转换为DATETIME格式的字符串
        return datetime.utcfromtimestamp(int(row)).strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        # 遇到无效的值时也返回None
        return None

# 处理可能的NaN值
df.fillna({'comment_mod_distinction': '', 'author': '', 'author_created': ''}, inplace=True)

# 连接到数据库
connection = pymysql.connect(host=host,
                             user=user,
                             password=password,
                             database=db,
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # 创建users表的SQL语句
        create_users_table_sql = """
        CREATE TABLE IF NOT EXISTS users (
            id INT AUTO_INCREMENT PRIMARY KEY,
            author VARCHAR(255) NOT NULL UNIQUE,
            author_comment_karma INT,
            author_post_karma INT,
            author_created DATETIME
        );
        """
        # 执行创建users表的SQL语句
        cursor.execute(create_users_table_sql)

        # 创建去重的DataFrame
        users_df = df[['author', 'author_comment_karma', 'author_post_karma', 'author_created']].drop_duplicates()
        users_df['author_created'] = users_df['author_created'].apply(convert_timestamp)

        # 遍历DataFrame中的每一行并插入数据到users表
        for index, row in users_df.iterrows():
            # 为每个字段再次检查NaN值，并将其转换为None
            row = row.where(pd.notnull(row), None)
            # 跳过author字段为空的行
            if row['author'] is None:
                continue
            # 构造INSERT语句
            insert_user_sql = """
            INSERT INTO users (author, author_comment_karma, author_post_karma, author_created)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                author_comment_karma = VALUES(author_comment_karma),
                author_post_karma = VALUES(author_post_karma),
                author_created = VALUES(author_created);
            """
            # 执行SQL语句
            cursor.execute(insert_user_sql, (row['author'], row['author_comment_karma'], row['author_post_karma'], row['author_created']))

    # 提交更改
    connection.commit()

    with connection.cursor() as cursor:
        # 创建posts表的SQL语句
        create_posts_table_sql = """
        CREATE TABLE IF NOT EXISTS posts (
            id INT AUTO_INCREMENT PRIMARY KEY,
            subreddit VARCHAR(255),
            title VARCHAR(255) NOT NULL UNIQUE,
            score INT,
            num_comments INT,
            body TEXT,
            created DATETIME,
            award VARCHAR(255)
        );
        """
        # 执行创建posts表的SQL语句
        cursor.execute(create_posts_table_sql)

        # 准备posts表的数据，去除重复的标题
        posts_df = df[['subreddit', 'post_title', 'post_score', 'post_num_comments', 'post_body', 'post_created', 'post_award']].drop_duplicates(subset=['post_title'])

        # 将Unix时间戳转换为MySQL的DATETIME格式
        posts_df['post_created'] = pd.to_datetime(posts_df['post_created'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')

        # 遍历DataFrame中的每一行并插入数据到posts表
        for index, row in posts_df.iterrows():
            # 构造插入帖子的SQL语句
            insert_post_sql = """
            INSERT INTO posts (subreddit, title, score, num_comments, body, created, award)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                score = VALUES(score),
                num_comments = VALUES(num_comments),
                body = VALUES(body),
                created = VALUES(created),
                award = VALUES(award);
            """
            # 执行SQL语句
            cursor.execute(insert_post_sql, (row['subreddit'], row['post_title'], row['post_score'], row['post_num_comments'], row['post_body'], row['post_created'], row['post_award']))
    connection.commit()
    
    with connection.cursor() as cursor:
        # 创建comments表的SQL语句
        create_comments_table_sql = """
        CREATE TABLE IF NOT EXISTS comments (
            id INT AUTO_INCREMENT PRIMARY KEY,
            comment_body TEXT,
            comment_score INT,
            comment_award VARCHAR(255),
            comment_mod_distinction VARCHAR(255),
            comment_created DATETIME,
            user_id INT,
            post_id INT,
            FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE SET NULL,
            FOREIGN KEY (post_id) REFERENCES posts(id) ON DELETE SET NULL
        );
        """
        # 执行创建comments表的SQL语句
        cursor.execute(create_comments_table_sql)

    # 提交更改
    connection.commit()
    
    # 以下代码需要根据您的CSV文件结构进行调整
    # 以下代码需要根据您的CSV文件结构进行调整
    with connection.cursor() as cursor:
        for index, row in df.iterrows():
            # 基于某些字段查询user_id和post_id
            cursor.execute("SELECT id FROM users WHERE author = %s", (row['author'],))
            user_result = cursor.fetchone()
            user_id = user_result['id'] if user_result else None
            
            cursor.execute("SELECT id FROM posts WHERE title = %s", (row['post_title'],))
            post_result = cursor.fetchone()
            post_id = post_result['id'] if post_result else None
            
            # 如果user_id和post_id都找到了，则插入评论数据
            if user_id and post_id:
                # 处理可能的NaN值
                comment_body = row['comment_body'] if not pd.isnull(row['comment_body']) else None
                comment_score = row['comment_score'] if not pd.isnull(row['comment_score']) else 0
                comment_award = row['comment_award'] if not pd.isnull(row['comment_award']) else None
                comment_mod_distinction = row['comment_mod_distinction'] if not pd.isnull(row['comment_mod_distinction']) else None
                comment_created = convert_timestamp(row['comment_created']) if not pd.isnull(row['comment_created']) else None
                
                insert_comment_sql = """
                INSERT INTO comments (comment_body, comment_score, comment_award, comment_mod_distinction, comment_created, user_id, post_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
                """
                cursor.execute(insert_comment_sql, (comment_body, comment_score, comment_award, comment_mod_distinction, comment_created, user_id, post_id))
    connection.commit()
    
finally:
    # 关闭数据库连接
    connection.close()
